# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
# Importar as libs
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

In [ ]:
# Ler o df
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [ ]:
## 1 - Ajustando modelo do log(renda)

# Criar variável resposta
df['log_renda'] = np.log(df['renda'])

# Definir fórmula incluindo todas as covariáveis menos data_ref, index, renda e log_renda
covs = [c for c in df.columns 
        if c not in ['data_ref', 'index', 'renda', 'log_renda']]
formula = 'log_renda ~ ' + ' + '.join(covs)

# Construir matrizes de desenho (dummies automáticas para qualitativas)
y, X = dmatrices(formula, data=df, return_type='dataframe')

# Ajustar regressão OLS
modelo_todos = sm.OLS(y, X).fit()

# Mostrar resumo
print(modelo_todos.summary())


                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     276.2
Date:                Wed, 25 Jun 2025   Prob (F-statistic):               0.00
Time:                        22:54:28   Log-Likelihood:                -13565.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12401   BIC:                         2.738e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

O modelo foi ajustado com todas as covariáveis disponíveis. As variáveis qualitativas foram automaticamente convertidas em dummies usando o patsy, mantendo a categoria mais frequente como referência (como o padrão do patsy já faz).
A maioria dos coeficientes faz sentido prático: por exemplo, escolaridade mais alta aumenta a renda, pessoas com filhos têm coeficiente negativo, e homens tendem a ter renda maior. Algumas variáveis não parecem muito relevantes, como posse_de_veiculo.

In [6]:
## 2 - Removendo variáveis insignificantes

# Extrair p-valores (sem Intercept)
pvals = modelo_todos.pvalues.drop('Intercept')

# Identificar variável de maior p-valor
var_remover = pvals.idxmax()
print(f'Variável a remover: {var_remover}')

# Ajustar novo modelo sem essa variável
covs2 = X.columns.drop(var_remover)
X2 = X[covs2]
modelo2 = sm.OLS(y, X2).fit()

# Resumo do novo modelo
print(modelo2.summary())

# Comparar indicadores
print('R²-ajustado:', modelo_todos.rsquared_adj, '→', modelo2.rsquared_adj)
print('AIC:        ', modelo_todos.aic,          '→', modelo2.aic)


Variável a remover: educacao[T.Secundário]
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.8
Date:                Wed, 25 Jun 2025   Prob (F-statistic):               0.00
Time:                        22:56:55   Log-Likelihood:                -13565.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

A variável menos significativa (com maior p-value) foi removida do modelo. Os principais indicadores se alteraram pouco: o R²-ajustado caiu levemente e o AIC aumentou um pouco, indicando que o modelo perdeu um pouco de qualidade.
Alguns coeficientes mudaram levemente, mas nada muito drástico, o que sugere que a variável removida não era tão relevante.

In [ ]:
### 3 - Removendo p-values menores que 0.5
# Copiar matrizes iniciais
X_iter = X.copy()
modelo_iter = modelo_todos

while True:
    # P-valores sem intercept
    p = modelo_iter.pvalues.drop('Intercept')
    p_max = p.max()
    
    # Condição de parada (todas p < 0.05)
    if p_max < 0.05:
        break
    
    # Remover variável de maior p-valor
    var = p.idxmax()
    print(f'Removendo {var} (p-val={p_max:.3f})')
    X_iter = X_iter.drop(columns=[var])
    
    # Reajustar modelo
    modelo_iter = sm.OLS(y, X_iter).fit()

# Resumo do modelo final
print(modelo_iter.summary())

# Indicadores finais
print('R²-ajustado final:', modelo_iter.rsquared_adj)
print('AIC final:        ', modelo_iter.aic)


Removendo educacao[T.Secundário] (p-val=0.817)
Removendo tipo_residencia[T.Com os pais] (p-val=0.710)
Removendo tipo_residencia[T.Governamental] (p-val=0.416)
Removendo tipo_residencia[T.Casa] (p-val=0.583)
Removendo tipo_residencia[T.Comunitário] (p-val=0.421)
Removendo tipo_renda[T.Bolsista] (p-val=0.408)
Removendo educacao[T.Pós graduação] (p-val=0.393)
Removendo educacao[T.Superior incompleto] (p-val=0.343)
Removendo tipo_renda[T.Pensionista] (p-val=0.234)
Removendo tipo_residencia[T.Estúdio] (p-val=0.183)
Removendo estado_civil[T.União] (p-val=0.168)
                            OLS Regression Results                            
Dep. Variable:              log_renda   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     492.7
Date:                Wed, 25 Jun 2025   Prob (F-statistic):               0.00
Time:                        22:57:25   Log

As remoções foram feitas de forma iterativa, sempre retirando a variável com maior p-value até todas as variáveis do modelo final serem estatisticamente significantes (p < 0.05).
O modelo final tem menos variáveis, é mais simples e apenas com variáveis relevantes. O R²-ajustado diminuiu um pouco, mas o AIC melhorou em relação ao modelo anterior.
Isso indica que, apesar de uma leve perda explicativa, o modelo ficou mais parcimonioso e robusto.